# 🛡️ Interactive Squad Simulation
Follow the directions at the top of each cell. After running the last cell, use the sliders and dropdowns below to run and visualize the simulation. (Test comment)

In [6]:
# If you are working in colab, use this block to clone the GitHub repo, install dependencies, and pull down the latest changes.
%cd /content/
!rm -rf SE3250-Spring2025-SquadSimulation
!git clone https://github.com/SuprMunchkin/SE3250-Spring2025-SquadSimulation.git
%cd SE3250-Spring2025-SquadSimulation
!pip install -r requirements.txt
%ls

/content
Cloning into 'SE3250-Spring2025-SquadSimulation'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 181 (delta 23), reused 25 (delta 14), pack-reused 133 (from 1)
Receiving objects: 100% (181/181), 1.88 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (73/73), done.
/content/SE3250-Spring2025-SquadSimulation
app.py   models/       requirements.txt       statdiagram.py
config/  __pycache__/  squad_simulator.ipynb  view/


In [9]:
# Import and setup
import sys
import yaml
import matplotlib.pyplot as plt
from ipywidgets import interact

# Custom imports
import os
sys.path.append("../models")
from models.squad_simulation import run_simulation
yaml_path = "config/simulation.yaml"
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

# Define interactive runner
def run_interactive_sim(blue_stock, hostile_stock, direction_deviation, armor_type, environment):
    """
    Run the interactive squad simulation and plot the results.

    Parameters:
        blue_stock (int): Number of blue units.
        hostile_stock (int): Number of hostile units.
        direction_deviation (int): Direction deviation in degrees.
        armor_type (str): Type of armor for blue units.
        environment (str): Simulation environment.
    """
    params = {
        "blue_stock": blue_stock,
        "hostile_stock": hostile_stock,
        "direction_deviation": direction_deviation,
        "armor_type": armor_type,
        "environment": environment
    }
    result = run_simulation(params)

    positions = result['positions']
    hostile_positions = result['hostile_positions']
    x_vals, y_vals = zip(*positions)
    hx_vals, hy_vals = zip(*hostile_positions)

    plt.figure(figsize=(8, 6))
    plt.plot(x_vals, y_vals, label='Blue Patrol Path', color='blue')
    plt.plot(hx_vals, hy_vals, label='Hostile Path', linestyle='--', color='red')
    plt.scatter(x_vals[0], y_vals[0], c='green', label='Start', zorder=5)
    plt.scatter(x_vals[-1], y_vals[-1], c='purple', label='End', zorder=5)
    plt.title("Squad Movement Simulation")
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

    print(f"👥 Blue Remaining: {result['blue']['stock']} / {params['blue_stock']}")
    print(f"🔴 Hostile Remaining: {result['hostile']['stock']} / {params['hostile_stock']}")

    # Print statements here can be used for troublshooting.
    #print("Blue positions: ", positions)
    #print("Hostile positions: ", hostile_positions)

# Create interactive widget interface
interact(
    run_interactive_sim,
    blue_stock=(1, 20),
    hostile_stock=(1, 40),
    direction_deviation=(0, 45, 5),
    armor_type=list(config['armor_profiles'].keys()),
    environment=list(config['threat_probs'].keys())
)

interactive(children=(IntSlider(value=10, description='blue_stock', max=20, min=1), IntSlider(value=20, descri…

<function __main__.run_interactive_sim(blue_stock, hostile_stock, direction_deviation, armor_type, environment)>